## Loading the Dataset

In [45]:
# importing necessary libraries
import pandas as pd

In [46]:
# defining relative path to file
file= './data/heart.csv'

# read file as pandas dataframe
df= pd.read_csv(file, sep=',', header= 0) # define seperator as ','

In [47]:
# validation that data was load correctly
df.head(10)

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope,HeartDisease
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up,0
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat,1
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up,0
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat,1
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up,0
5,39,M,NAP,120,339,0,Normal,170,N,0.0,Up,0
6,45,F,ATA,130,237,0,Normal,170,N,0.0,Up,0
7,54,M,ATA,110,208,0,Normal,142,N,0.0,Up,0
8,37,M,ASY,140,207,0,Normal,130,Y,1.5,Flat,1
9,48,F,ATA,120,284,0,Normal,120,N,0.0,Up,0


In [48]:
# check the columns of the dataframe
columns= df.columns
print(columns)
print(f'Number of columns: {len(columns)}')

Index(['Age', 'Sex', 'ChestPainType', 'RestingBP', 'Cholesterol', 'FastingBS',
       'RestingECG', 'MaxHR', 'ExerciseAngina', 'Oldpeak', 'ST_Slope',
       'HeartDisease'],
      dtype='object')
Number of columns: 12


The column above shows the columns of the data frame. In total the dataframe has 12 columns. The column 'HeartDisease' is the target column that is supposed.
In the following step the divided in feature columns and the target column.

In [49]:
# divide dataframe into featues and labels
df_train = df.iloc[:, :len(df.columns)-1]
df_label= df.iloc[:,-1]

In [50]:
df_train.head()

,Age,Sex,ChestPainType,RestingBP,Cholesterol,FastingBS,RestingECG,MaxHR,ExerciseAngina,Oldpeak,ST_Slope
0,40,M,ATA,140,289,0,Normal,172,N,0.0,Up
1,49,F,NAP,160,180,0,Normal,156,N,1.0,Flat
2,37,M,ATA,130,283,0,ST,98,N,0.0,Up
3,48,F,ASY,138,214,0,Normal,108,Y,1.5,Flat
4,54,M,NAP,150,195,0,Normal,122,N,0.0,Up


In [51]:
df_label.head()

0    0
1    1
2    0
3    1
4    0
Name: HeartDisease, dtype: int64

In [52]:
# Check the quality of the data
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 918 entries, 0 to 917
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Age             918 non-null    int64  
 1   Sex             918 non-null    object 
 2   ChestPainType   918 non-null    object 
 3   RestingBP       918 non-null    int64  
 4   Cholesterol     918 non-null    int64  
 5   FastingBS       918 non-null    int64  
 6   RestingECG      918 non-null    object 
 7   MaxHR           918 non-null    int64  
 8   ExerciseAngina  918 non-null    object 
 9   Oldpeak         918 non-null    float64
 10  ST_Slope        918 non-null    object 
 11  HeartDisease    918 non-null    int64  
dtypes: float64(1), int64(6), object(5)
memory usage: 86.2+ KB


The datasets consists of 918 observations. On first sight it seems to be that there a no missing values.

In [53]:
# Show the class distribution of the target column
df.HeartDisease.value_counts(normalize= True)

HeartDisease
1    0.553377
0    0.446623
Name: proportion, dtype: float64

The target class is distrbuted in 55% observations that related ti a heart disease and 44% without heart disease. This gets quite close to an almost equal distribution. So the target class is well balanced.

In [54]:
# shows descriptive statistics for numeric columns
df.describe()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,HeartDisease
count,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000,918.000000
mean,53.510893,132.396514,198.799564,0.233115,136.809368,0.887364,0.553377
std,9.432617,18.514154,109.384145,0.423046,25.460334,1.066570,0.497414
min,28.000000,0.000000,0.000000,0.000000,60.000000,-2.600000,0.000000
25%,47.000000,120.000000,173.250000,0.000000,120.000000,0.000000,0.000000
50%,54.000000,130.000000,223.000000,0.000000,138.000000,0.600000,1.000000
75%,60.000000,140.000000,267.000000,0.000000,156.000000,1.500000,1.000000
max,77.000000,200.000000,603.000000,1.000000,202.000000,6.200000,1.000000


The describe function of the dataframe gives a statical overview of the numeric columns. Firstly there are no missing values. Secondly it can be noted that the scale of the data has big differences. This becomes clear comparing values like 'Age' and 'Cholesterol'. This shows scaling gets necessary.

In [55]:
# Checking for null values
df_train.isnull().sum()

Age               0
Sex               0
ChestPainType     0
RestingBP         0
Cholesterol       0
FastingBS         0
RestingECG        0
MaxHR             0
ExerciseAngina    0
Oldpeak           0
ST_Slope          0
dtype: int64

In [56]:
# Scaling numeric values
num_cols= ['Age', 'RestingBP', 'Cholesterol', 'FastingBS', 'MaxHR', 'Oldpeak']

In [57]:
# Following the numeric coloumns of the datafram will be scaled
for col in num_cols:
    df_train[col] = (df_train[col] - df_train[col].mean()) / df_train[col].std()


In [58]:
# Transform categorical variable into dummy variables; source: https://pandas.pydata.org/docs/reference/api/pandas.get_dummies.html
cat_cols= ['Sex', 'ChestPainType','RestingECG','ExerciseAngina', 'ST_Slope']
df_train= pd.get_dummies(df_train, columns=cat_cols)
df_train.head()

,Age,RestingBP,Cholesterol,FastingBS,MaxHR,Oldpeak,Sex_F,Sex_M,ChestPainType_ASY,ChestPainType_ATA,ChestPainType_NAP,ChestPainType_TA,RestingECG_LVH,RestingECG_Normal,RestingECG_ST,ExerciseAngina_N,ExerciseAngina_Y,ST_Slope_Down,ST_Slope_Flat,ST_Slope_Up
0,-1.432359,0.410685,0.824621,-0.551041,1.382175,-0.831979,False,True,False,True,False,False,False,True,False,True,False,False,False,True
1,-0.478223,1.490940,-0.171867,-0.551041,0.753746,0.105606,True,False,False,False,True,False,False,True,False,True,False,False,True,False
2,-1.750404,-0.129442,0.769768,-0.551041,-1.524307,-0.831979,False,True,False,True,False,False,False,False,True,True,False,False,False,True
3,-0.584238,0.302660,0.138964,-0.551041,-1.131539,0.574398,True,False,True,False,False,False,False,True,False,False,True,False,True,False
4,0.051853,0.950812,-0.034736,-0.551041,-0.581664,-0.831979,False,True,False,False,True,False,False,True,False,True,False,False,False,True


In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
import numpy as np

In [ ]:
def build_model(learning_rate= .1, loss= 'binary_crossentropy', activation='relu'):
    model= models.Sequential()
    model.add(layers.Dense())